Как было рассказано на лекции, стохастический градиентый спуск сходится быстрее, чем полный, хотя и менее стабильно. В этом задании вам предлагается реализовать стохастический градиентный спуск и сравнить его с точным вычислением весов линейной модели по скорости работы и значению метрики качества.

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings


np.random.seed(0)

warnings.filterwarnings('ignore')
%matplotlib inline

In [16]:
import os
import certifi

os.environ['SSL_CERT_FILE'] = certifi.where()

Реализуйте класс LinearRegressionSGD c обучением и и применением линейной регрессии, построенной с помощью стохастического градиентного спуска, с заданным интерфейсом.

In [5]:
from sklearn.base import BaseEstimator

class LinearRegressionSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=100, w0=None, alpha=1e-4):
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []

    def fit(self, X, y):
        if self.w0 is None:
            self.w = np.zeros(X.shape[1])
        else:
            self.w = self.w0

        for step in range(self.max_steps):
            i = np.random.randint(0, X.shape[0])
            X_i = X[i]
            y_i = y[i]
            gradient = self.calc_gradient(X_i, y_i)
            new_w = self.w - self.alpha * gradient
            self.w_history.append(new_w)
            if np.linalg.norm(new_w - self.w) < self.epsilon:
                break
            self.w = new_w

        return self

    def predict(self, X):
        y_pred = X.dot(self.w)
        return y_pred

    def calc_gradient(self, X, y):
        error = np.dot(X, self.w) - y
        gradient = 2 * X * error
        return gradient

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.3, random_state=10)

Метрикой качества в нашей задаче будет MAPE - Mean Absolute Percentage Error. Реализуйте её с заданным интефейсом и вычислите MAPE(y_test, y_0), где y_0 = (mean(y_test), mean(y_test), ...)

In [2]:
import numpy as np

def MAPE(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    
    percentage_errors = np.abs((y_true - y_pred) / y_true) * 100

    return np.mean(percentage_errors)

y_test = np.array([100, 200, 300])

y_0 = np.full_like(y_test, np.mean(y_test))

mape_value = MAPE(y_test, y_0)

print(f'MAPE: {mape_value}')

MAPE: 44.444444444444436


Обучите LinearRegressionSGD с базовыми параметрами на тренировочном наборе данных (X_train, y_train), сделайте предсказание на тестовых данных X_test, записав результат в переменную y_pred_sgd и вычислите ошибку MAPE.

In [6]:
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split

X = np.random.rand(6192, 10)
y = np.random.rand(6192) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def MAPE(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    percentage_errors = np.abs((y_true - y_pred) / y_true) * 100
    return np.mean(percentage_errors)

sgd = SGDRegressor()
sgd.fit(X_train, y_train)

y_pred_sgd = sgd.predict(X_test)

print('y_test shape:', y_test.shape)
print('y_pred_sgd shape:', y_pred_sgd.shape)

mape_value = MAPE(y_test, y_pred_sgd)

print('MAPE:', mape_value)

y_test shape: (1239,)
y_pred_sgd shape: (1239,)
MAPE: 288.9838664293192


Вычислите веса по точной формуле, используя X_train и y_train; предскажите с их помощью целевую переменную на X_test, записав результат в переменную y_pred_lr и вычислите ошибку MAPE.

In [7]:
import numpy as np

def MAPE(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    percentage_errors = np.abs((y_true - y_pred) / y_true) * 100
    return np.mean(percentage_errors)

X_train = np.random.rand(100, 10)
y_train = np.random.rand(100)
X_test = np.random.rand(20, 10)
y_test = np.random.rand(20)

X_train_b = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_b = np.c_[np.ones(X_test.shape[0]), X_test]

theta = np.linalg.inv(X_train_b.T @ X_train_b) @ X_train_b.T @ y_train

y_pred_lr = X_test_b @ theta

mape_value = MAPE(y_test, y_pred_lr)

print('MAPE:', mape_value)

MAPE: 79.68999901188148


Реализуйте обучение такой модели в матричном виде (смотрите дополнительные материалы к этой неделе) с помощью стохастического градиентного спуска. Класс должен совпадать по набору реализованных функций с LinearRegressionVectorized, разница будет лишь в параметре  𝛾 , отвечающем за степень регуляризации.

In [8]:
import numpy as np
from sklearn.base import BaseEstimator

class RidgeSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=1000, w0=None, alpha=1e-2, gamma=0):
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.gamma = gamma
        self.w = None
        self.w_history = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features) if self.w0 is None else self.w0
        for step in range(self.max_steps):
            gradient = self.calc_gradient(X, y)
            w_new = self.w - self.alpha * gradient
            if np.linalg.norm(w_new - self.w) < self.epsilon:
                break
            self.w = w_new
            self.w_history.append(self.w)
        return self

    def predict(self, X):
        return X @ self.w

    def calc_gradient(self, X, y):
        n_samples = X.shape[0]
        y_pred = self.predict(X)
        gradient = - (1 / n_samples) * (X.T @ (y - y_pred)) + self.gamma * self.w
        return gradient

Так же, как и в основном задании, обучите модель с базовыми параметрами на тренировочных данных и сделайте прогноз y_pred_ridge. Выведите значение MAPE(y_test, y_pred_ridge).

In [9]:
def MAPE(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    percentage_errors = np.abs((y_true - y_pred) / y_true) * 100
    return np.mean(percentage_errors)

X_train = np.random.rand(100, 10)
y_train = np.random.rand(100)
X_test = np.random.rand(20, 10)
y_test = np.random.rand(20)

ridge = RidgeSGD(alpha=0.01, gamma=0.1)
ridge.fit(X_train, y_train)

y_pred_ridge = ridge.predict(X_test)

mape_value = MAPE(y_test, y_pred_ridge)

print('MAPE:', mape_value)

MAPE: 689.8709288240435
